<a href="https://colab.research.google.com/github/Narusi/Python-Kurss/blob/master/ss_com_Dzivoklu_datu_nolasisana_un_analize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ss.com datu ielase un analīze

## Moduļu imports

In [1]:
#Private module
import ssWebScraper as sws

#Public packages
import matplotlib.pyplot as plt
import seaborn as sn
import plotly.express as px

import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn import neighbors

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



## Datu ielāde

In [ ]:
districts = sws.gatherSubCats(initialLink='https://www.ss.com/en/real-estate/flats/riga/')
ipasumi = sws.readPostList(districts, page_n=60, verbose=True)

Processing Centre
Processing Agenskalns
Processing Aplokciems
Processing Beberbeki
Processing Bergi
Processing Bierini
Processing Bolderaya


In [ ]:
ipasumi.info()

### Datu glabāšana

In [ ]:
ipasumi = sws.saveToDB(ipasumi, tableName='PropertiesRAW',  
                       uniqCols = ['District','Street Name','Street No.', 'Rooms', 
                                   'Size','Floor', 'Max. Floor', 'Project'])

In [ ]:
ipasumi = sws.loadFromDB(tableName='PropertiesRAW', dbName = 'miniSS.db')
ipasumi.head(5)

## Datu attīrīšana

In [ ]:
ipasumi.info()

In [ ]:
print('Pirms tīrīšanas')
ipasumi[['Price of sqm','Alt. Price of sqm']].hist(figsize=(16,4))
plt.show();

altPrInex = ipasumi['Alt. Price of sqm'] >= ipasumi['Price of sqm'].max()
ipasumi.loc[altPrInex, 'Alt. Price of sqm'] = ipasumi.loc[altPrInex, 'Price of sqm']

#cleanDF = ipasumi.loc[ipasumi['Alt. Price of sqm'] <= ipasumi['Price of sqm'].max()]
cleanDF = ipasumi.copy()
cleanDF = cleanDF.loc[cleanDF['Size'] <= 1000]
cleanDF = cleanDF.loc[cleanDF['Floor'] <= cleanDF['Max. Floor']]

print('Pēc tīrīšanas')
cleanDF[['Price of sqm','Alt. Price of sqm']].hist(figsize=(16,4))
plt.show();

In [ ]:
cleanDF['AVG_Price_SQM'] = 0.0

attrNames = ['District','Project','Max. Floor','Street Name']
for r in cleanDF.index:
    perfMatch = cleanDF.copy()
    for i in range(4):
        attrValue = cleanDF.loc[r, attrNames[i]]
        perfMatch = perfMatch.loc[perfMatch[attrNames[i]] == attrValue, :]
        
        if perfMatch.count().min() <= 10: break;
    
    cleanDF.loc[r, 'AVG_Price_SQM'] = perfMatch.drop(r, axis=0).loc[:,'Alt. Price of sqm'].mean()    

In [ ]:
cleanDF['Project_AVG'] = 0.0
cleanDF['District_AVG'] = 0.0

cleanDF['Project_AVG'] = cleanDF.groupby('Project')['Alt. Price of sqm'].transform(np.mean)
cleanDF['District_AVG'] = cleanDF.groupby('District')['Alt. Price of sqm'].transform(np.mean)

cleanDF.info()

In [ ]:

for c in cleanDF.describe().columns:
    for row in cleanDF.index:
        similarDF = cleanDF.loc[np.logical_and(cleanDF['District'] == cleanDF.loc[row,'District'],
                                               cleanDF['Project']== cleanDF.loc[row,'Project'])]
        similarDF = similarDF[similarDF['Deal Type'] == cleanDF.loc[row,'Deal Type']]
        
        avg = similarDF[c].mean()
        std = similarDF[c].std()        
        
        if similarDF[c].count() > 3 and c not in ['Rooms','Size','Floor','Max. Floor']:
            if abs(cleanDF.loc[row,c] - avg) > 10 * std:
                if cleanDF.loc[row,c] > maxim: print('Bigger than 99th percentile');
                if abs(cleanDF.loc[row,c] - avg) > 10 * std: print('Further than 10 standard deviations from mean');

                print(c)
                print(cleanDF.loc[row])
                if similarDF.loc[similarDF['Street Name'] == cleanDF.loc[row,'Street Name'], c].count() > 3:
                    cleanDF.loc[row,c] = similarDF.loc[similarDF['Street Name'] == cleanDF.loc[row,'Street Name'], c].mean()
                else:
                    cleanDF.loc[row,c] = avg
        else:
            if c == 'Rooms' and cleanDF.loc[row,c] > 10: cleanDF = cleanDF.drop(row, axis=0);
            if c == 'Floor' and cleanDF.loc[row,c] > 100: cleanDF = cleanDF.drop(row, axis=0);
            if c == 'Max. Floor' and cleanDF.loc[row,c] > 100: cleanDF = cleanDF.drop(row, axis=0);
            if c == 'Price of sqm' and cleanDF.loc[row,c] > abs(cleanDF.loc[row,c] - avg) > 2 * std: 
                cleanDF = cleanDF.drop(row, axis=0);
                continue
            if c == 'Alt. Price of sqm' and cleanDF.loc[row,c] > 100: 
                cleanDF = cleanDF.drop(row, axis=0);
                continue
            if c == 'Total Price' and cleanDF.loc[row,c] > 100: 
                cleanDF = cleanDF.drop(row, axis=0);
                continue
            if c == 'Alt. Price' and cleanDF.loc[row,c] > 100: 
                cleanDF = cleanDF.drop(row, axis=0);
                continue
                
            if cleanDF.loc[row,c]/cleanDF.loc[row,'Project_AVG'] > 5 or cleanDF.loc[row,c]/cleanDF.loc[row,'District_AVG'] > 5:
                try:
                    if cleanDF.loc[row,c] != np.nan:
                        cleanDF.loc[row,c] = cleanDF.loc[row, 'AVG_Price_SQM']
                    else:
                        cleanDF.loc[row,c] = cleanDF.loc[row, ['Project_AVG','District_AVG']].mean()
                except:
                    cleanDF.loc[row,c] = cleanDF.loc[row, ['Project_AVG','District_AVG']].mean()
            
cleanDF = cleanDF.dropna()

## Datu analīze

In [ ]:
pd.plotting.scatter_matrix(cleanDF, figsize=(14,14))
plt.show();

In [ ]:
fig = px.bar(cleanDF.groupby('District')[['Alt. Price of sqm','AVG_Price_SQM']].mean(), barmode='group')
fig.show();

fig = px.bar(cleanDF.groupby('District')[['Alt. Price of sqm','AVG_Price_SQM']].std(), barmode='group')
fig.show();

fig = px.bar(cleanDF.groupby('District')[['Alt. Price of sqm']].count(), barmode='group')
fig.show();

In [ ]:
fig = px.scatter(cleanDF.dropna(), x="District", y ="Rooms", size="AVG_Price_SQM", color="Project", 
                 log_x=False, size_max=45)
fig.show()

In [ ]:
for proj in cleanDF.loc[:, 'Project'].drop_duplicates():
    tempDF = cleanDF.loc[cleanDF['Project']==proj, :]
    tempDF['ID'] = tempDF['Link'] + tempDF['Street No.']
    
    tempDF = tempDF.drop_duplicates('ID').pivot(index='ID', columns='District', values='Alt. Price of sqm')
    
    fig = px.box(tempDF, color='District', title=proj)
    fig.show();


In [ ]:
cleanDF[['District', 'Project', 'AVG_Price_SQM']].dropna().sort_values(by=['District','Project'])
distN = cleanDF.groupby(['District'])['Size'].count().dropna()
projN = cleanDF.groupby(['Project'])['Size'].count().dropna()

presentDF = cleanDF.copy()
for i in cleanDF['District'].drop_duplicates():
    if i not in distN[distN>distN.quantile(0.25)].index:
        presentDF = presentDF[presentDF['District'] != i]
    
for i in cleanDF['Project'].drop_duplicates():
    if i not in projN[projN>projN.quantile(0.25)].index:
        presentDF = presentDF[presentDF['Project'] != i]
    
for distr in presentDF['District'].drop_duplicates().sort_values():
    fig = px.bar(presentDF.loc[presentDF['District'] == distr].groupby('Project')['AVG_Price_SQM'].mean(),
                title = distr + ' vidējās kvm cenas pēc ēkas projekta')
    fig.show();

In [ ]:
presentDF[presentDF['Project']=='New'].groupby('District')['AVG_Price_SQM'].describe().round(2)

### Īres dati

In [ ]:
cleanDF[cleanDF['Deal Type']=='SELL'].hist(figsize=(16,14))
plt.show();            

cleanDF[cleanDF['Deal Type']=='RENT'].describe()

In [ ]:
cleanDF['Relative_Floor'] = cleanDF['Floor'].sub(1)/cleanDF['Max. Floor']
cleanDF.loc[cleanDF['Max. Floor'] == 1,'Relative_Floor'] = 1.

for c in cleanDF.drop(['Deal Type','Comment','Link','Street Name','Street No.',
                      'Alt. Price of sqm','Post Date','Price of sqm','Total Price',
                      'Alt. Price'], axis=1).columns:
    
    if cleanDF[c].dtype == 'object' or c == 'Rooms':
        fig = px.box(cleanDF[cleanDF['Deal Type']=='RENT'], x=c, y="Alt. Price of sqm", points="all", color=c)
    elif c == 'Relative_Floor':
        fig = px.histogram(cleanDF[cleanDF['Deal Type']=='RENT'], x=c, y="Alt. Price of sqm", histfunc='avg')
    else:
        fig = px.scatter(cleanDF[cleanDF['Deal Type']=='RENT'], x=c, y="Alt. Price of sqm", color=c)
    fig.show()

In [ ]:
cleanDF[cleanDF['Deal Type']=='RENT'].describe().round(2)

In [ ]:
features = ['Rooms','Size', 'Project_AVG', 'District_AVG']
fig = px.scatter_matrix(
    cleanDF[cleanDF['Deal Type']=='RENT'],
    dimensions=features + ['Alt. Price of sqm','Floor', 'Price of sqm'],
    color='Project'
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
pca = PCA(n_components=4)

data_scaled = cleanDF.loc[cleanDF['Deal Type']=='RENT',features+['Alt. Price of sqm']]

components = pca.fit_transform(data_scaled.dropna())

labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

pca_rent_results = pd.DataFrame(pca.components_, 
                                columns=data_scaled.columns,
                                index = labels.values()).round(3)
pca_rent_results

In [ ]:
fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4),
    color=cleanDF.loc[cleanDF['Deal Type']=='RENT','Project']
)

fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
X = data_scaled.drop('Alt. Price of sqm', axis=1)
y = cleanDF.loc[data_scaled.index, 'Price of sqm']

linRegCorr = pd.DataFrame(index=X.columns)
linRegCorr['Rent'] = 0.0
for c in X.columns:
    linRegCorr.loc[c, 'Rent'] = np.corrcoef(X[c],y)[1][0]
    
linRegCorr

In [ ]:
reg = LinearRegression().fit(X, y)
rent_coefficients = pd.Series(reg.coef_, index=X.columns)

px.bar(rent_coefficients, title='Regresijas koeficienti').show()

print('Score', reg.score(X, y))
print('Intercept', reg.intercept_)

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

In [ ]:
knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
knn_model = knn.fit(X[:2*int(len(y)/3)], y[:2*int(len(y)/3)])
knn_model.score(X[int(len(y)/3):], y[int(len(y)/3):])

### Pārdošanas darījumi

In [ ]:
cleanDF[cleanDF['Deal Type']=='SELL'].hist(figsize=(16,14))
plt.show();

cleanDF[cleanDF['Deal Type']=='SELL'].describe().round(2)

In [ ]:
for c in cleanDF.drop(['Deal Type','Comment','Link','Street Name','Street No.',
                      'Alt. Price of sqm','Post Date','Price of sqm','Total Price',
                      'Alt. Price'], axis=1).columns:
    
    if cleanDF[c].dtype == 'object' or c == 'Rooms':
        fig = px.box(cleanDF[cleanDF['Deal Type']=='SELL'], x=c, y="Alt. Price of sqm", points="all", color=c)
    elif c == 'Relative_Floor':
        fig = px.histogram(cleanDF[cleanDF['Deal Type']=='SELL'], x=c, y="Alt. Price of sqm", histfunc='avg')
    else:
        fig = px.scatter(cleanDF[cleanDF['Deal Type']=='SELL'], x=c, y="Alt. Price of sqm", color=c)
    fig.show()

In [ ]:
pca = PCA(n_components=4)

data_scaled = cleanDF.loc[cleanDF['Deal Type']=='SELL',features+['Alt. Price of sqm']]

components = pca.fit_transform(data_scaled.dropna())

labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

pca_rent_results = pd.DataFrame(pca.components_, 
                                columns=data_scaled.columns,
                                index = labels.values()).round(3)
pca_rent_results

In [ ]:
fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(4)    
)

fig.update_traces(diagonal_visible=False)
fig.show()

In [ ]:
X = data_scaled.drop('Alt. Price of sqm', axis=1)
y = cleanDF.loc[data_scaled.index, 'Price of sqm']

linRegCorr['Sell'] = 0.0
for c in X.columns:
    linRegCorr.loc[c,'Sell'] = np.corrcoef(X[c],y)[1][0]

linRegCorr

In [ ]:
knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
knn_model = knn.fit(X[:2*int(len(y)/3)], y[:2*int(len(y)/3)])
knn_model.score(X[int(len(y)/3):], y[int(len(y)/3):])

In [ ]:
reg = LinearRegression().fit(X, y)
print('R2:', reg.score(X, y))
rent_coefficients = pd.Series(reg.coef_, index=X.columns)
px.bar(rent_coefficients, title='Regresijas koeficienti').show()
print('Intercept', reg.intercept_)

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

### Visi darījumi

In [ ]:
pca = PCA(n_components=4)

data_scaled = cleanDF.loc[:,features+['Alt. Price of sqm']]

components = pca.fit_transform(data_scaled.dropna())

labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

pca_rent_results = pd.DataFrame(pca.components_, 
                                columns=data_scaled.columns,
                                index = labels.values()).round(3)
pca_rent_results

In [ ]:
X = data_scaled.drop('Alt. Price of sqm', axis=1)
y = cleanDF.loc[data_scaled.index, 'Price of sqm']

linRegCorr['Overall'] = 0.0
for c in X.columns:
    linRegCorr.loc[c,'Overall'] = np.corrcoef(X[c],y)[1][0]

linRegCorr

In [ ]:
model = sm.OLS(y, X)
results = model.fit()

results.summary()

In [ ]:
knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
knn_model = knn.fit(X[:2*int(len(y)/3)], y[:2*int(len(y)/3)])
knn_model.score(X[int(len(y)/3):], y[int(len(y)/3):])

# Prezentācijai

## Kavadrātmetru cenas un to vidējās vērtības

In [ ]:
for dealType in cleanDF['Deal Type'].drop_duplicates():
    px.scatter(cleanDF[cleanDF['Deal Type']==dealType], 
               x='AVG_Price_SQM', y='Alt. Price of sqm', 
               color='AVG_Price_SQM', title=dealType).show()
    korrKof = cleanDF.loc[cleanDF['Deal Type']==dealType, ['AVG_Price_SQM','Alt. Price of sqm']].corr().iloc[1,0]
    print(f'Korelācijas koeficients: {round(100*korrKof,2)}%')
    
    fig = plt.figure(figsize=(14,3))
    sn.heatmap(cleanDF[cleanDF['Deal Type']==dealType].corr().loc[['Price of sqm','Alt. Price of sqm', 'Total Price', 
                                                              'Alt. Price'],
                                                             ['Rooms','Size','Floor','Relative_Floor',
                                                             'Project_AVG','District_AVG']], annot=True)
    plt.show()

In [ ]:
px.scatter_3d(cleanDF, x='District', y='Project', z='Alt. Price of sqm', 
              color='Deal Type', symbol='Deal Type', opacity=0.85).show()

## k-Tuvāko kaimiņu (KNN) metode vs Lineārā regresija 

In [ ]:
postCounts = pd.crosstab(index=cleanDF['Post Date'], columns=cleanDF['Deal Type'])
fig = px.bar(postCounts.sum(), title='Kopējais novērotais sludinājumu sakits pēc darījuma tipa')
fig.update_layout(showlegend=False)
fig.show()


In [ ]:
rent_X = cleanDF.loc[cleanDF['Deal Type']=='RENT',['Rooms','Size','Floor','Max. Floor',
                                                   'Project_AVG','District_AVG','Relative_Floor']]
sell_X = cleanDF.loc[cleanDF['Deal Type']=='SELL',['Rooms','Size','Floor','Max. Floor',
                                                   'Project_AVG','District_AVG','Relative_Floor']]
total_X = cleanDF.loc[:, ['Rooms','Size','Floor','Max. Floor','Project_AVG','District_AVG','Relative_Floor']]

rent_y = cleanDF.loc[cleanDF['Deal Type']=='RENT', 'Alt. Price of sqm']
sell_y = cleanDF.loc[cleanDF['Deal Type']=='SELL', 'Alt. Price of sqm']
total_y = cleanDF.loc[:, 'Alt. Price of sqm']

In [ ]:
rent_knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
rent_model = rent_knn.fit(rent_X[:2*int(len(rent_y)/3)], rent_y[:2*int(len(rent_y)/3)])
rent_R2 = rent_model.score(rent_X[int(len(rent_y)/3):], rent_y[int(len(rent_y)/3):])

sell_knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
sell_model = sell_knn.fit(sell_X[:2*int(len(sell_y)/3)], sell_y[:2*int(len(sell_y)/3)])
sell_R2 = sell_model.score(sell_X[int(len(sell_y)/3):], sell_y[int(len(sell_y)/3):])

total_knn = neighbors.KNeighborsRegressor(n_neighbors=10, weights='distance')
total_model = sell_knn.fit(total_X[:2*int(len(total_y)/3)], total_y[:2*int(len(total_y)/3)])
total_R2 = total_model.score(total_X[int(len(total_y)/3):], total_y[int(len(total_y)/3):])

print(f'Īres dzīvokļiem tuvāko kaimiņu metodes R2: {round(100*rent_R2,2)}%')
print(f'Pārdošanas dzīvokļiem tuvāko kaimiņu metodes R2: {round(100*sell_R2,2)}%')
print(f'Visiem dzīvokļiem tuvāko kaimiņu metodes R2: {round(100*total_R2,2)}%')

In [ ]:
rent_X = rent_X.drop(['Floor','Max. Floor','Relative_Floor'], axis=1)
sell_X = sell_X.drop(['Floor','Max. Floor','Relative_Floor'], axis=1)
total_X = total_X.drop(['Floor','Max. Floor','Relative_Floor'], axis=1)

In [ ]:
rent_LR_model = sm.OLS(rent_y, rent_X).fit()
sell_LR_model = sm.OLS(sell_y, sell_X).fit()
total_LR_model = sm.OLS(total_y, total_X).fit()

print('ĪRES DZĪVOKĻI'.center(85))
print(rent_LR_model.summary())
print()

print('PĀRDOŠANAS DZĪVOKĻI'.center(85))
print(sell_LR_model.summary())
print()

print('VISI DZĪVOKĻI'.center(85))
print(total_LR_model.summary())

## Kvadrātmetra cena pēc rajona un projekta

In [ ]:
fig = px.box(cleanDF, x='District', y="Alt. Price of sqm", color='Project')
fig.show()

# Potenciālās dzīvokļa cenas noteikšana

In [ ]:
import ipywidgets as widgets

project = widgets.Dropdown(options=cleanDF['Project'].drop_duplicates().sort_values().to_list(), description='Projekts:')
district = widgets.Dropdown(options=cleanDF['District'].drop_duplicates().sort_values().to_list(),description='Rajons:')
dealTypeLV = widgets.Dropdown(options=['Pirkt','Īrēt'], description='Darījums:')

project

In [ ]:
district

In [ ]:
dealTypeLV

In [ ]:
rooms = int(input('Ievadiet dzīvokļa istabu skaitu: '))
size = float(input('Ievadiet dzīvokļa izmēru kvadrātmetros: '))

In [ ]:
model = {'Pirkt':sell_LR_model,'Īrēt':rent_LR_model}[dealTypeLV.value]
dealType = {'Pirkt':'SELL','Īrēt':'RENT'}[dealTypeLV.value]

project_AVG = cleanDF.loc[np.logical_and(cleanDF['Deal Type']==dealType,
                                         cleanDF['Project']==project.value),'Project_AVG'].mean()
district_AVG = cleanDF.loc[np.logical_and(cleanDF['Deal Type']==dealType,
                                          cleanDF['District']==district.value),'District_AVG'].mean()

if dealType == 'RENT':
    price = size*np.dot(model.params, pd.Series([rooms,size,project_AVG, district_AVG]))/120
    print(f'Dzīvokļa potenciālā īres cena: {round(price,1)} EUR.')
else:
    price = size * np.dot(model.params, pd.Series([rooms,size,project_AVG, district_AVG]))
    print(f'Dzīvokļa potenciālā pirkuma cena: {round(price,1)} EUR.')